https://github.com/brianhie/scanorama

In [1]:
import scanorama
import anndata as ad
import scanpy as sc

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/intervaltree/intervaltree.py:37: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  class IntervalTree(collections.MutableSet):


In [2]:
sc.set_figure_params(dpi=100, frameon=False, color_map='Reds')
sc.logging.print_header()

/home/icb/yuge.ji/miniconda3/envs/py37/lib/python3.7/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)
2022-01-25 17:41:36.037873: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-25 17:41:36.037938: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


scanpy==1.8.2 anndata==0.7.6 umap==0.5.2 numpy==1.20.3 scipy==1.5.3 pandas==1.3.4 scikit-learn==1.0.2 statsmodels==0.11.1 python-igraph==0.8.3 leidenalg==0.8.3 pynndescent==0.5.5


In [3]:
adata = sc.read('/storage/groups/ml01/workspace/yuge.ji/covid-perturbation/data/211015_annot_patient_integrated.h5ad')
adata

AnnData object with n_obs × n_vars = 300353 × 18115
    obs: 'identifier', 'patient', 'name', 'barcode_round', 'sample', 'viral_counts', 'treatment', 'treatment_virus', 'infected', 'n_counts', 'n_genes', 'percent_mito', 'percent_viral', 'leiden', 'final_bcs', 'batch', 'celltype_coarse', 'cell_type', 'infection_label', '_scvi_batch', '_scvi_labels', 'celltype_fine'
    var: 'used_for_scvi'
    uns: '_scvi', 'cell_type_colors', 'celltype_coarse_colors', 'celltype_fine_colors', 'hvg', 'infected_colors', 'leiden', 'leiden_colors', 'neighbors', 'patient_colors', 'rank_genes_groups', 'treatment_colors', 'umap'
    obsm: 'X_pca', 'X_scVI', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [4]:
adatas = [adata[adata.obs.patient == p] for p in adata.obs.patient.unique()]

In [5]:
%%time
# Integration and batch correction.
corrected = scanorama.correct_scanpy(adatas, return_dimred=True)

Found 18115 genes among all datasets
[[0.         0.82201433 0.01079512 0.00631457]
 [0.         0.         0.12545504 0.08247544]
 [0.         0.         0.         0.50119778]
 [0.         0.         0.         0.        ]]
Processing datasets (0, 1)
Processing datasets (2, 3)
Processing datasets (1, 2)
CPU times: user 9h 22min 16s, sys: 41min 29s, total: 10h 3min 45s
Wall time: 1h 24min 14s


In [10]:
adata

AnnData object with n_obs × n_vars = 300353 × 18115
    obs: 'identifier', 'patient', 'name', 'barcode_round', 'sample', 'viral_counts', 'treatment', 'treatment_virus', 'infected', 'n_counts', 'n_genes', 'percent_mito', 'percent_viral', 'leiden', 'final_bcs', 'batch', 'celltype_coarse', 'cell_type', 'infection_label', '_scvi_batch', '_scvi_labels', 'celltype_fine'
    var: 'used_for_scvi'
    uns: '_scvi', 'cell_type_colors', 'celltype_coarse_colors', 'celltype_fine_colors', 'hvg', 'infected_colors', 'leiden', 'leiden_colors', 'neighbors', 'patient_colors', 'rank_genes_groups', 'treatment_colors', 'umap'
    obsm: 'X_pca', 'X_scVI', 'X_umap'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [11]:
%%time
new_adata = ad.concat(corrected)

CPU times: user 16.6 s, sys: 1min 47s, total: 2min 3s
Wall time: 2min 5s


In [ ]:
sc.write('/storage/groups/ml01/workspace/yuge.ji/covid-perturbation/data/211015_annot_patient_integrated_scanorama.h5ad', adata, compression='gzip')

Add back in metadata.

In [16]:
new_adata = new_adata[adata.obs.index, :]

In [ ]:
new_adata.uns = adata.uns.copy()
for field in ['X_pca', 'X_scVI', 'X_umap']:
    new_adata.obsm[field] = adata.obsm[field]

In [ ]:
new_adata.layers['counts'] = adata.layers['counts']

In [ ]:
sc.write('/storage/groups/ml01/workspace/yuge.ji/covid-perturbation/data/211015_annot_patient_integrated_scanorama.h5ad', adata, compression='gzip')

In [1]:
sc.pl.umap(new_adata, color='cell_type')